<a href="https://colab.research.google.com/github/ManiacMaxo/Python/blob/master/ai/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import random
from shutil import copyfile

In [2]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2020-11-01 15:34:11--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.33.180.114, 2600:1408:2000:1b1::e59, 2600:1408:2000:18f::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.33.180.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M   112MB/s    in 6.8s    

2020-11-01 15:34:18 (115 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [3]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

12501
12501


In [4]:
try:
    os.mkdir('/tmp/cats-v-dogs/')

    os.mkdir('/tmp/cats-v-dogs/train')
    os.mkdir('/tmp/cats-v-dogs/train/cats')
    os.mkdir('/tmp/cats-v-dogs/train/dogs')

    os.mkdir('/tmp/cats-v-dogs/test')
    os.mkdir('/tmp/cats-v-dogs/test/cats')
    os.mkdir('/tmp/cats-v-dogs/test/dogs')
except OSError:
    pass

In [5]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = set(os.listdir(SOURCE))
    train = set(random.sample(files, round(len(files) * SPLIT_SIZE)))
    test = files - train

    for f in train:
        if os.path.getsize(SOURCE + f) == 0:
            print(f'{f} is zero length, so ignoring')
            continue
        copyfile(SOURCE + f, TRAINING + f)
    
    for f in test:
        if os.path.getsize(SOURCE + f) == 0:
            print(f'{f} is zero length, so ignoring')
            continue
        copyfile(SOURCE + f, TESTING + f)

In [6]:
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/train/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/test/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/train/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/test/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

666.jpg is zero length, so ignoring
11702.jpg is zero length, so ignoring


In [7]:
print(len(os.listdir('/tmp/cats-v-dogs/train/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/train/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/test/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/test/dogs/')))

11251
11250
1249
1250


In [8]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
img_size = 300

In [18]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(516, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
TRAINING_DIR = '/tmp/cats-v-dogs/train'
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(img_size, img_size),
    class_mode='binary'
)

VALIDATION_DIR = '/tmp/cats-v-dogs/test'
validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(img_size, img_size),
    class_mode='binary'
)

Found 22499 images belonging to 2 classes.
Found 2499 images belonging to 2 classes.


In [20]:
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/15
203/704 [=======>......................] - ETA: 1:09 - loss: 34.9083 - accuracy: 0.5686

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

704/704 [==============================] - 107s 152ms/step - loss: 10.9316 - accuracy: 0.5809 - val_loss: 0.6572 - val_accuracy: 0.6218
Epoch 2/15
704/704 [==============================] - 106s 150ms/step - loss: 0.7134 - accuracy: 0.6623 - val_loss: 0.5883 - val_accuracy: 0.7015
Epoch 3/15
704/704 [==============================] - 106s 150ms/step - loss: 0.5498 - accuracy: 0.7434 - val_loss: 0.5314 - val_accuracy: 0.7331
Epoch 4/15
704/704 [==============================] - 107s 152ms/step - loss: 0.4646 - accuracy: 0.7931 - val_loss: 0.5388 - val_accuracy: 0.7475
Epoch 5/15
704/704 [==============================] - 106s 151ms/step - loss: 0.4032 - accuracy: 0.8242 - val_loss: 0.5747 - val_accuracy: 0.7515
Epoch 6/15
704/704 [==============================] - 108s 153ms/step - loss: 0.3498 - accuracy: 0.8603 - val_loss: 0.5385 - val_accuracy: 0.7495
Epoch 7/15
704/704 [==============================] - 108s 154ms/step - loss: 0.3043 - accuracy: 0.8897 - val_loss: 0.7805 - val_accur

In [24]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(img_size, img_size))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0] > 0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

Saving _111434467_gettyimages-1143489763.jpg to _111434467_gettyimages-1143489763 (2).jpg
[6.6658e-11]
_111434467_gettyimages-1143489763.jpg is a cat
